# CycleGAN

## Import Libraries

In [0]:
import numpy as np
import pandas as pd
import tensorflow as tf

### Utils.py

Utility functions to process data set: load test and train data sets, read images, save images, crop images, transform/inverse transform images


In [0]:
"""
Some codes from https://github.com/Newmu/dcgan_code
"""
from __future__ import division
import math
import pprint
import scipy.misc
import numpy as np
import copy
#try:
    #_imread = scipy.misc.imread
#except AttributeError:
from imageio import imread as _imread
from imageio import imwrite as _imwrite
#from PIL import Image as PILImage
from skimage import transform as skimage_transform

pp = pprint.PrettyPrinter()

get_stddev = lambda x, k_h, k_w: 1/math.sqrt(k_w*k_h*x.get_shape()[-1])

# -----------------------------
# new added functions for cyclegan
class ImagePool(object):
    def __init__(self, maxsize=50):
        self.maxsize = maxsize
        self.num_img = 0
        self.images = []

    def __call__(self, image):
        if self.maxsize <= 0:
            return image
        if self.num_img < self.maxsize:
            self.images.append(image)
            self.num_img += 1
            return image
        if np.random.rand() > 0.5:
            idx = int(np.random.rand()*self.maxsize)
            tmp1 = copy.copy(self.images[idx])[0]
            self.images[idx][0] = image[0]
            idx = int(np.random.rand()*self.maxsize)
            tmp2 = copy.copy(self.images[idx])[1]
            self.images[idx][1] = image[1]
            return [tmp1, tmp2]
        else:
            return image

def load_test_data(image_path, fine_size=256):
    img = imread(image_path)
    img = skimage_transform.resize(img, (fine_size, fine_size))
    img = img/127.5 - 1
    return img

def load_train_data(image_path, load_size=286, fine_size=256, is_testing=False):
    img_A = imread(image_path[0])
    img_B = imread(image_path[1])
    if not is_testing:
        img_A = skimage_transform.resize(img_A, (load_size, load_size))
        img_B = skimage_transform.resize(img_B, (load_size, load_size))

        h1 = int(np.ceil(np.random.uniform(1e-2, load_size-fine_size)))
        w1 = int(np.ceil(np.random.uniform(1e-2, load_size-fine_size)))
        img_A = img_A[h1:h1+fine_size, w1:w1+fine_size]
        img_B = img_B[h1:h1+fine_size, w1:w1+fine_size]

        if np.random.random() > 0.5:
            img_A = np.fliplr(img_A)
            img_B = np.fliplr(img_B)
    else:
        img_A = skimage_transform.resize(img_A, (fine_size, fine_size))
        img_B = skimage_transform.resize(img_B, (fine_size, fine_size))

    img_A = img_A/127.5 - 1.
    img_B = img_B/127.5 - 1.

    img_AB = np.concatenate((img_A, img_B), axis=2)
    # img_AB shape: (fine_size, fine_size, input_c_dim + output_c_dim)
    return img_AB

# -----------------------------

def get_image(image_path, image_size, is_crop=True, resize_w=64, is_grayscale = False):
    return transform(imread(image_path, is_grayscale), image_size, is_crop, resize_w)

def save_images(images, size, image_path):
    return imsave(inverse_transform(images), size, image_path)

def imread(path, is_grayscale = False):
    if (is_grayscale):
        return _imread(path, as_gray=True).astype(np.float)
    else:
        return _imread(path, pilmode='RGB').astype(np.float)

def merge_images(images, size):
    return inverse_transform(images)

def merge(images, size):
    h, w = images.shape[1], images.shape[2]
    img = np.zeros((h * size[0], w * size[1], 3))
    for idx, image in enumerate(images):
        i = idx % size[1]
        j = idx // size[1]
        img[j*h:j*h+h, i*w:i*w+w, :] = image

    return img

def imsave(images, size, path):
    return _imwrite(path, merge(images, size))

def center_crop(x, crop_h, crop_w,
                resize_h=64, resize_w=64):
  if crop_w is None:
    crop_w = crop_h
  h, w = x.shape[:2]
  j = int(round((h - crop_h)/2.))
  i = int(round((w - crop_w)/2.))
  return skimage_transform.resize(
      x[j:j+crop_h, i:i+crop_w], (resize_h, resize_w))

def transform(image, npx=64, is_crop=True, resize_w=64):
    # npx : # of pixels width/height of image
    if is_crop:
        cropped_image = center_crop(image, npx, resize_w=resize_w)
    else:
        cropped_image = image
    return np.array(cropped_image)/127.5 - 1.

def inverse_transform(images):
    return (images+1.)/2.


### ops.py

Defines operations which are later to be used to define generator and discrimnator networks. Includes, batch_norm, instance_norm, conv2d, deconv2d, lrelu and linear functions.

In [0]:
import math
import numpy as np
import tensorflow as tf
import tensorflow.contrib.slim as slim
from tensorflow.python.framework import ops

# from utils import *

##### New Helper Functions

# weight and bais wrappers
def weight_variable(name, shape):
    """
    Create a weight variable with appropriate initialization
    :param name: weight name
    :param shape: weight shape
    :return: initialized weight variable
    """
    initer = tf.truncated_normal_initializer(stddev=0.01)
    return tf.get_variable('W_' + name,
                           dtype=tf.float32,
                           shape=shape,
                           initializer=initer)

def bias_variable(name, shape):
    """
    Create a bias variable with appropriate initialization
    :param name: bias variable name
    :param shape: bias variable shape
    :return: initialized bias variable
    """
    initial = tf.constant(0., shape=shape, dtype=tf.float32)
    return tf.get_variable('b_' + name,
                           dtype=tf.float32,
                           initializer=initial)
 

def fc_layer(x, num_units, name, use_relu=True):
    """
    Create a fully-connected layer
    :param x: input from previous layer
    :param num_units: number of hidden units in the fully-connected layer
    :param name: layer name
    :param use_relu: boolean to add ReLU non-linearity (or not)
    :return: The output array
    """
    in_dim = x.get_shape()[1]
    W = weight_variable(name, shape=[in_dim, num_units])
    b = bias_variable(name, [num_units])
    layer = tf.matmul(x, W)
    layer += b
    if use_relu:
        layer = tf.nn.relu(layer)
    return layer


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



### module.py

Used to define descriminator network, generator network (with and without residual block), and three loss functions.

In [0]:
from __future__ import division
import tensorflow as tf
# from ops import *
# from utils import *

# discriminator network
def discriminator(pca_vector, options, reuse=False, name="discriminator"):

    with tf.variable_scope(name):
        # image is 256 x 256 x input_c_dim
        if reuse:
            tf.get_variable_scope().reuse_variables()
        else:
            assert tf.get_variable_scope().reuse is False
            
        output_dimension = pca_vector.shape[1]

        h0 = fc_layer(pca_vector, 128, name='d_h0', use_relu=True)
        # h0 is (128 x 128 x self.df_dim)
        h1 = fc_layer(h0, 128, name='d_h1', use_relu=True)
        # h1 is (64 x 64 x self.df_dim*2)
        h2 = fc_layer(h1, 128, name='d_h2', use_relu=True)
        # h2 is (32x 32 x self.df_dim*4)
        h3 = fc_layer(h2, 128, name='d_h3', use_relu=True)
        # h3 is (32 x 32 x self.df_dim*8)
        h4 = fc_layer(h3, 128, name='d_h4', use_relu=True)
        # h4 is (32 x 32 x 1)
        return h4

# generator network without residual block
def generator(pca_vector, options, reuse=False, name="generator"):

    with tf.variable_scope(name):
        # pca_vector is ((k=32 < m=number of coordinates x 2) x input_c_dim)
        if reuse:
            tf.get_variable_scope().reuse_variables()
        else:
            assert tf.get_variable_scope().reuse is False
            
        output_dimension = pca_vector.shape[1]

        # pca_vector is ((k=32 < m=number of coordinates x 2) x input_c_dim)
        e1 = fc_layer(pca_vector, 64, name='g_e1', use_relu=True)
        # e1 is (128 x 128 x self.gf_dim) 1048576
        e2 = fc_layer(e1, 128, name='g_e2', use_relu=True)
        # e2 is (64 x 64 x self.gf_dim*2) 524288
        e3 = fc_layer(e2, 256, name='g_e3', use_relu=True)
        # e3 is (32 x 32 x self.gf_dim*4) 262144
        e4 = fc_layer(e3, 512, name='g_e4', use_relu=True)
        # e4 is (16 x 16 x self.gf_dim*8) 131072
        e5 = fc_layer(e4, 256, name='g_e5', use_relu=True)
        # e5 is (8 x 8 x self.gf_dim*8) 32768
        e6 = fc_layer(e5, 128, name='g_e6', use_relu=True)
        # e6 is (4 x 4 x self.gf_dim*8) 8192
        e7 = fc_layer(e6, 64, name='g_e7', use_relu=True)
        # e7 is (2 x 2 x self.gf_dim*8) 2048
        e8 = fc_layer(e7, output_dimension, name='g_e8', use_relu=False)
        # e8 is (1 x 1 x self.gf_dim*8) 512

        return e8

    
# loss function
def abs_criterion(in_, target):
    return tf.reduce_mean(tf.abs(in_ - target))

## loss function
def mae_criterion(in_, target):
    return tf.reduce_mean((in_-target)**2)

## loss function
def sce_criterion(logits, labels):
    return tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=labels))


## loss function
def cosine_distance(real_1, real_2, fake_2):
    # real_1 minus mean of real_1 for all y in Y
    
    # test = tf.fill(tf.shape(real_1)[:-1], 1)
    # test = tf.ones(real_1.shape[0])
    A = real_1 - tf.matmul(tf.fill(tf.shape(real_1)[:-1], 1).reshape(-1,1), real_1.mean(axis=0).reshape(1,-1))

    # fake_2 minus mean of X for all y in Y
    B = fake_2 - tf.matmul(tf.fill(tf.shape(real_1)[:-1], 1).reshape(-1,1), real_2.mean(axis=0).reshape(1,-1))

    # calculate cosine distance matrix
    cosine_distance_matrix = tf.multiply(tf.ones((real_1.shape[0],1)) - cosine_similarity(A, B, dense_output=True),
                                         tf.eye(real_1.shape[0]))
    
    # calculate loss
    cos_dist = cosine_distance_matrix.sum()
    
    return cos_dist
    


### Model.py

In [0]:
from __future__ import division
import os
import time
from glob import glob
import tensorflow as tf
import numpy as np
from collections import namedtuple

# from module import *
# from utils import *

# COMMENT: 
class cyclegan(object):
    def __init__(self, sess, args):
        # COMMENT:
        self.sess = sess
        self.batch_size = args.batch_size
        
        self.num_components = args.pca_components

        self.L1_lambda = args.L1_lambda
        self.dataset_dir = args.dataset_dir
        self.discriminator = discriminator
        self.generator = generator

        # COMMENT:
        if args.use_lsgan:
            self.criterionGAN = mae_criterion
        else:
            self.criterionGAN = sce_criterion

        # COMMENT:
        OPTIONS = namedtuple('OPTIONS', 'batch_size image_size \
                              gf_dim df_dim is_training')
        self.options = OPTIONS._make((args.batch_size, args.pca_components,
                                      args.ngf, args.ndf,
                                      args.phase == 'train'))
        
        # COMMENT:
        self._build_model()
        
        # COMMENT:
        self.saver = tf.train.Saver()
        
        # COMMENT:
        self.pool = ImagePool(args.max_size)

    def _build_model(self):
        """
        COMMENT:
        
        """
        
        # COMMENT:
        self.real_A = tf.placeholder(tf.float32,
                                     [None, self.num_components],
                                     name='real_A')
        self.real_B= tf.placeholder(tf.float32,
                                    [None, self.num_components],
                                    name='real_B')

        # COMMENT:
        self.fake_B = self.generator(self.real_A, self.options, False, name="generatorA2B")
        self.fake_A_ = self.generator(self.fake_B, self.options, False, name="generatorB2A")
        self.fake_A = self.generator(self.real_B, self.options, True, name="generatorB2A")
        self.fake_B_ = self.generator(self.fake_A, self.options, True, name="generatorA2B")

        # COMMENT:
        self.DB_fake = self.discriminator(self.fake_B, self.options, reuse=False, name="discriminatorB")
        self.DA_fake = self.discriminator(self.fake_A, self.options, reuse=False, name="discriminatorA")


        # COMMENT:
        self.g_loss_a2b = self.criterionGAN(self.DB_fake, tf.ones_like(self.DB_fake)) \
            + self.L1_lambda * abs_criterion(self.real_A, self.fake_A_) \
            + self.L1_lambda * abs_criterion(self.real_B, self.fake_B_)
        self.g_loss_b2a = self.criterionGAN(self.DA_fake, tf.ones_like(self.DA_fake)) \
            + self.L1_lambda * abs_criterion(self.real_A, self.fake_A_) \
            + self.L1_lambda * abs_criterion(self.real_B, self.fake_B_)
        
        self.char_loss_a2b = cosine_distance(self.real_A, self.real_B, self.fake_B)
        self.char_loss_b2a = cosine_distance(self.real_B, self.real_A, self.fake_A)
        
        self.g_loss = self.criterionGAN(self.DA_fake, tf.ones_like(self.DA_fake)) \
            + self.criterionGAN(self.DB_fake, tf.ones_like(self.DB_fake)) \
            + self.L1_lambda * abs_criterion(self.real_A, self.fake_A_) \
            + self.L1_lambda * abs_criterion(self.real_B, self.fake_B_) \
            + self.char_loss_a2b \
            + self.char_loss_b2a
        
        
        # COMMENT:
        self.fake_A_sample = tf.placeholder(tf.float32,
                                            [None, self.num_components], name='fake_A_sample')
        self.fake_B_sample = tf.placeholder(tf.float32,
                                            [None, self.num_components], name='fake_B_sample')

        # COMMENT:
        self.DB_real = self.discriminator(self.real_B, self.options, reuse=True, name="discriminatorB")
        self.DA_real = self.discriminator(self.real_A, self.options, reuse=True, name="discriminatorA")
        self.DB_fake_sample = self.discriminator(self.fake_B_sample, self.options, reuse=True, name="discriminatorB")
        self.DA_fake_sample = self.discriminator(self.fake_A_sample, self.options, reuse=True, name="discriminatorA")

        # COMMENT:
        self.db_loss_real = self.criterionGAN(self.DB_real, tf.ones_like(self.DB_real))
        self.db_loss_fake = self.criterionGAN(self.DB_fake_sample, tf.zeros_like(self.DB_fake_sample))
        self.db_loss = (self.db_loss_real + self.db_loss_fake) / 2
        self.da_loss_real = self.criterionGAN(self.DA_real, tf.ones_like(self.DA_real))
        self.da_loss_fake = self.criterionGAN(self.DA_fake_sample, tf.zeros_like(self.DA_fake_sample))
        self.da_loss = (self.da_loss_real + self.da_loss_fake) / 2
        self.d_loss = self.da_loss + self.db_loss

        # COMMENT:
        self.g_loss_a2b_sum = tf.summary.scalar("g_loss_a2b", self.g_loss_a2b)
        self.g_loss_b2a_sum = tf.summary.scalar("g_loss_b2a", self.g_loss_b2a)
        self.g_loss_sum = tf.summary.scalar("g_loss", self.g_loss)
        # COMMENT:
        self.g_sum = tf.summary.merge([self.g_loss_a2b_sum, self.g_loss_b2a_sum, self.g_loss_sum])

        # COMMENT:
        self.db_loss_sum = tf.summary.scalar("db_loss", self.db_loss)
        self.da_loss_sum = tf.summary.scalar("da_loss", self.da_loss)
        self.d_loss_sum = tf.summary.scalar("d_loss", self.d_loss)
        self.db_loss_real_sum = tf.summary.scalar("db_loss_real", self.db_loss_real)
        self.db_loss_fake_sum = tf.summary.scalar("db_loss_fake", self.db_loss_fake)
        self.da_loss_real_sum = tf.summary.scalar("da_loss_real", self.da_loss_real)
        self.da_loss_fake_sum = tf.summary.scalar("da_loss_fake", self.da_loss_fake)

        # COMMENT:
        self.d_sum = tf.summary.merge(
            [self.da_loss_sum, self.da_loss_real_sum, self.da_loss_fake_sum,
             self.db_loss_sum, self.db_loss_real_sum, self.db_loss_fake_sum,
             self.d_loss_sum]
        )

        
        # COMMENT:
        self.test_A = tf.placeholder(tf.float32,
                                     [None, self.num_components], name='test_A')
        self.test_B = tf.placeholder(tf.float32,
                                     [None, self.num_components], name='test_B')
        self.testB = self.generator(self.test_A, self.options, True, name="generatorA2B")
        self.testA = self.generator(self.test_B, self.options, True, name="generatorB2A")

        t_vars = tf.trainable_variables()

        
        # COMMENT:
        self.d_vars = [var for var in t_vars if 'discriminator' in var.name]
        self.g_vars = [var for var in t_vars if 'generator' in var.name]
        for var in t_vars: print(var.name)

    def train(self, args):
        """Train cyclegan"""
        # COMMENT:
        self.lr = tf.placeholder(tf.float32, None, name='learning_rate')

        # COMMENT:
        self.d_optim = tf.train.AdamOptimizer(self.lr, beta1=args.beta1) \
            .minimize(self.d_loss, var_list=self.d_vars)
        self.g_optim = tf.train.AdamOptimizer(self.lr, beta1=args.beta1) \
            .minimize(self.g_loss, var_list=self.g_vars)

        # COMMENT:
        init_op = tf.global_variables_initializer()

        # COMMENT:
        self.sess.run(init_op)

        # COMMENT:
        self.writer = tf.summary.FileWriter("./logs", self.sess.graph)

        # COMMENT:
        counter = 1
        start_time = time.time()

        # COMMENT:
        if args.continue_train:
            if self.load(args.checkpoint_dir):
                print(" [*] Load SUCCESS")
            else:
                print(" [!] Load failed...")

        # COMMENT:
        for epoch in range(args.epoch):

            # COMMENT:
            dataA = glob('./datasets/{}/*.*'.format(self.dataset_dir + '/trainA'))
            dataB = glob('./datasets/{}/*.*'.format(self.dataset_dir + '/trainB'))
            np.random.shuffle(dataA)
            np.random.shuffle(dataB)
            batch_idxs = min(min(len(dataA), len(dataB)), args.train_size) // self.batch_size

            # COMMENT:
            lr = args.lr if epoch < args.epoch_step else args.lr*(args.epoch-epoch)/(args.epoch-args.epoch_step)

            # COMMENT:
            for idx in range(0, batch_idxs):
                # COMMENT:
                batch_files = list(zip(dataA[idx * self.batch_size:(idx + 1) * self.batch_size],
                                       dataB[idx * self.batch_size:(idx + 1) * self.batch_size]))

                # COMMENT:
                batch_images = [load_train_data(batch_file, args.load_size, args.pca_components) for batch_file in batch_files]

                # COMMENT:
                batch_images = np.array(batch_images).astype(np.float32)

                # COMMENT:
                fake_A, fake_B, _, summary_str = self.sess.run(
                    [self.fake_A, self.fake_B, self.g_optim, self.g_sum],
                    feed_dict={self.real_data: batch_images, 
                               self.lr: lr})
                self.writer.add_summary(summary_str, counter)
                [fake_A, fake_B] = self.pool([fake_A, fake_B])

                # COMMENT:
                _, summary_str = self.sess.run(
                    [self.d_optim, self.d_sum],
                    feed_dict={self.real_data: batch_images,
                               self.fake_A_sample: fake_A,
                               self.fake_B_sample: fake_B,
                               self.lr: lr})
                self.writer.add_summary(summary_str, counter)

                # COMMENT:
                counter += 1
                print(("Epoch: [%2d] [%4d/%4d] time: %4.4f" % (
                    epoch, idx, batch_idxs, time.time() - start_time)))

                # COMMENT:
                if np.mod(counter, args.print_freq) == 1:
                    self.sample_model(args.sample_dir, epoch, idx)

                # COMMENT:
                if np.mod(counter, args.save_freq) == 2:
                    self.save(args.checkpoint_dir, counter)

    def save(self, checkpoint_dir, step):
        model_name = "cyclegan.model"
        model_dir = "%s_%s" % (self.dataset_dir, self.num_components)
        checkpoint_dir = os.path.join(checkpoint_dir, model_dir)

        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)

        self.saver.save(self.sess,
                        os.path.join(checkpoint_dir, model_name),
                        global_step=step)

    def load(self, checkpoint_dir):
        print(" [*] Reading checkpoint...")

        model_dir = "%s_%s" % (self.dataset_dir, self.num_components)
        checkpoint_dir = os.path.join(checkpoint_dir, model_dir)

        ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
        if ckpt and ckpt.model_checkpoint_path:
            ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
            self.saver.restore(self.sess, os.path.join(checkpoint_dir, ckpt_name))
            return True
        else:
            return False

    def sample_model(self, sample_dir, epoch, idx):
        dataA = glob('./datasets/{}/*.*'.format(self.dataset_dir + '/testA'))
        dataB = glob('./datasets/{}/*.*'.format(self.dataset_dir + '/testB'))
        np.random.shuffle(dataA)
        np.random.shuffle(dataB)
        batch_files = list(zip(dataA[:self.batch_size], dataB[:self.batch_size]))
        sample_images = [load_train_data(batch_file, is_testing=True) for batch_file in batch_files]
        sample_images = np.array(sample_images).astype(np.float32)

        fake_A, fake_B = self.sess.run(
            [self.fake_A, self.fake_B],
            feed_dict={self.real_data: sample_images}
        )
        save_images(fake_A, [self.batch_size, 1],
                    './{}/A_{:02d}_{:04d}.jpg'.format(sample_dir, epoch, idx))
        save_images(fake_B, [self.batch_size, 1],
                    './{}/B_{:02d}_{:04d}.jpg'.format(sample_dir, epoch, idx))

    def test(self, args):
        """Test cyclegan"""
        init_op = tf.global_variables_initializer()
        self.sess.run(init_op)
        if args.which_direction == 'AtoB':
            sample_files = glob('./datasets/{}/*.*'.format(self.dataset_dir + '/testA'))
        elif args.which_direction == 'BtoA':
            sample_files = glob('./datasets/{}/*.*'.format(self.dataset_dir + '/testB'))
        else:
            raise Exception('--which_direction must be AtoB or BtoA')

        if self.load(args.checkpoint_dir):
            print(" [*] Load SUCCESS")
        else:
            print(" [!] Load failed...")

        # write html for visual comparison
        index_path = os.path.join(args.test_dir, '{0}_index.html'.format(args.which_direction))
        index = open(index_path, "w")
        index.write("<html><body><table><tr>")
        index.write("<th>name</th><th>input</th><th>output</th></tr>")

        out_var, in_var = (self.testB, self.test_A) if args.which_direction == 'AtoB' else (
            self.testA, self.test_B)

        for sample_file in sample_files:
            print('Processing image: ' + sample_file)
            sample_image = [load_test_data(sample_file, args.pca_components)]
            sample_image = np.array(sample_image).astype(np.float32)
            image_path = os.path.join(args.test_dir,
                                      '{0}_{1}'.format(args.which_direction, os.path.basename(sample_file)))
            fake_img = self.sess.run(out_var, feed_dict={in_var: sample_image})
            save_images(fake_img, [1, 1], image_path)
            index.write("<td>%s</td>" % os.path.basename(image_path))
            index.write("<td><img src='%s'></td>" % (sample_file if os.path.isabs(sample_file) else (
                '..' + os.path.sep + sample_file)))
            index.write("<td><img src='%s'></td>" % (image_path if os.path.isabs(image_path) else (
                '..' + os.path.sep + image_path)))
            index.write("</tr>")
        index.close()


### main.py

In [0]:
tf.reset_default_graph()


class Args():
    dataset_dir = 'horse2zebra'
    epoch = 200
    epoch_step = 100
    batch_size = 1
    train_size = 1e8
    load_size = 286
    pca_components = 32
    ngf = 64
    ndf = 64
    lr = 0.0002
    beta1 = 0.5
    which_direction = 'AtoB'
    phase = 'train'
    save_freq = 1000
    print_freq = 100
    continue_train = False
    checkpoint_dir = './checkpoint'
    sample_dir = './sample'
    test_dir = './test'
    L1_lambda = 10.0
    use_lsgan = True
    max_size = 50

args = Args()

tfconfig = tf.ConfigProto(allow_soft_placement=True)
tfconfig.gpu_options.allow_growth = True
with tf.Session(config=tfconfig) as sess:
    model = cyclegan(sess, args)
    #model.train(args) 
    #if args.phase == 'train' \
     #   else model.test(args)


AttributeError: ignored